In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import datetime as dt
import json
import requests
import schedule
import time
import tweepy
def run_algo():
    url = 'https://api.binance.com/api/v3/klines'
    symbol = 'ETHUSDT'
    interval = '1h'
    
    start = str(int(dt.datetime(2023,1,1).timestamp()*1000))
    end = str(int(dt.datetime(2023,2,1).timestamp()*1000))
    par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end, 'limit': 1000}
    eth = pd.DataFrame(json.loads(requests.get(url, params= par).text))
    eth.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
    eth.drop(columns=['open', 'high', 'low', 'volume', 'close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'], inplace = True) #we will only use close price
    eth.index = [dt.datetime.fromtimestamp(x/1000.0) for x in eth.datetime]
    eth=eth.astype(float)

    #algo
    eth['10d-ma'] = eth['close'].rolling(10).mean().shift()
    eth['20d-ma'] = eth['close'].rolling(20).mean().shift()

    #entries
    eth['signal'] = np.where(eth['10d-ma'] > eth['20d-ma'], 1, 0) #long signal
    eth['signal'] = np.where(eth['10d-ma'] < eth['20d-ma'], -1, eth['signal']) #short signal
    eth.dropna(inplace = True)
    eth['entry'] =  eth.signal.diff()

    plt.rcParams['figure.figsize'] = 19, 9
    plt.grid(True, alpha = .3)
    plt.plot(eth.iloc[-180:]['close'], label = 'eth')
    plt.plot(eth.iloc[-180:]['10d-ma'], label = '10d-ma')
    plt.plot(eth.iloc[-180:]['20d-ma'], label = '20d-ma')
    plt.plot(eth[-180:].loc[eth.entry == 2].index, eth[-180:]['close'][eth.entry == 2], '^',
             color = 'g', markersize = 8)
    plt.plot(eth[-180:].loc[eth.entry == -2].index, eth[-180:]['close'][eth.entry == -2], 'v',
             color = 'r', markersize = 8)
    plt.legend(loc=2);
    plt.savefig("eth_ma_1h.jpg")
    plt.clf()

    ############################################## V1

    if(eth['signal'][-1] != eth['signal'][-2]):
        if(eth['signal'][-1] == 1):
            pos = 'eth/usdt long  @ '
            price = str(eth['close'][-1])
            algo = '\n(moving avg 1h)'
            text = pos+price+algo
        else:
            pos='eth/usdt short  @ '
            price=str(eth['close'][-1])
            entry=pos+price
            algo = '\n(moving avg 1h)'
            text=pos+price+algo
            
        #posting on twitter
        # Enter your API credentials
        consumer_key = 'key'
        consumer_secret = 'secret'
        access_token = 'token'
        access_token_secret = 'token_secret'

        # Authenticate with the Twitter API using your API credentials
        auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)

        # Connect to the Twitter API using the authenticate handler
        api = tweepy.API(auth)

        # Open the image file
        image_path = 'eth_ma_1h.jpg'
        image = open(image_path, 'rb')

        # Set the status update text
        status_text = text

        # Upload the image to Twitter
        upload_response = api.media_upload(image_path)
        media_id = upload_response.media_id_string

        # Publish the tweet with the image and status update
        status = api.update_status(status=status_text, media_ids=[media_id]) 
    
schedule.every().hour.at(":00").do(run_algo)
while True:
    schedule.run_pending()
    time.sleep(0.99)